In [ ]:
import msnoise.api as ms
import numpy as np
import matplotlib.pyplot as plt

In [2]:
path_ini = 'D:\\MEMOIRE\\MSNoise\\ablation\\db.ini'
database = ms.connect(inifile=path_ini)

In [3]:
NET = ms.get_networks(database)
STA = ['BJK','DJK','HUS','VOT','JOK','KSK','URH']

In [4]:
for net in NET :
    for sta in STA :
        ms.mark_data_availability(database, net, sta, 'N')

In [4]:
ms.update_config(database,'startdate', '2019-01-01')

In [7]:
ms.get_networks(database)

['SS']

In [6]:
ms.update_filter(database,ref=2,low=1.0,high=2.0,mwcs_low=1.0,mwcs_high=2.0, mwcs_wlen=120, 
                     mwcs_step=60 , rms_threshold=0, used=True)

In [15]:
ms.get_filters(database, all=True)

[<msnoise.msnoise_table_def.declare_tables.<locals>.Filter at 0x1fe06d1ad60>,
 <msnoise.msnoise_table_def.declare_tables.<locals>.Filter at 0x1fe06d1a910>]

In [18]:
stt = ms.get_stations(database, all=True)

In [20]:
ms.get_interstation_distance(stt[1],stt[2],coordinates='DEG')

19.366583213567928